# TK Module 3

In [2]:
# load libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('./USO.csv')

In [16]:
df.head()

,date,open,high,low,close,unadjustedclose,volume,log_close
0,20060410,68.25,68.50,67.67,68.02,68.02,38779,4.219802
1,20060411,68.32,68.39,67.30,68.20,68.20,12971,4.222445
2,20060412,68.22,68.81,67.81,67.84,67.84,12483,4.217152
3,20060413,67.50,68.99,67.40,68.82,68.82,5607,4.231494
4,20060417,69.20,69.90,68.68,69.79,69.79,9177,4.245491


In [12]:
df.columns = df.columns.str.replace(' ', '') 
df.columns = df.columns.str.lower()

In [4]:
def getWeights_FFD(d,thres):
    w,k=[1.],1
    while True:
        w_=-w[-1]/k*(d-k+1)
        if abs(w_)<thres:break
        w.append(w_);k+=1
    return np.array(w[::-1]).reshape(-1,1)

In [5]:
def fracDiff_FFD(series,d,thres=1e-5):
    # Constant width window (new solution)
    w=getWeights_FFD(d,thres)
    df={}
    for name in series.columns:
        df_= pd.Series(series[name].values,index=series.index).fillna(method='ffill').dropna()
        x = pd.Series(0,index=df_.index)
        for k in range(w.shape[0]):
            x = x+w[k,0]*df_.shift(-k)
    #df[name]=x.dropna().copy(deep=True)
    df[name]=x.shift(k).copy(deep=True)
    df=pd.concat(df,axis=1)
    return df

In [6]:
def findMinD(series):
    for d in np.linspace(0,1,11):
        df_=fracDiff_FFD(series,d,thres=0.01).dropna()
        res=adfuller(df_.iloc[:,0].values,maxlag=1,regression='c',autolag=None)
        if (res[0]<=res[4]['5%']):
            return d
    return 1.0

## Question 1

**find the minimum fractional dimension d that will turn this price series into a stationary series. (30 points)**



In [19]:
log_mind = findMinD(df[['log_close']])
norm_mind = findMinD(df[['close']])

print(f'log Min Dimensions: {log_mind}')
print(f'norm Min Dimensions: {norm_mind}')

log Min Dimensions: 0.5
norm Min Dimensions: 0.4


## Question 2

**Using the d you found in part 1 to fractionally differentiate the USO daily price series, use adfuller from statsmodels.tsa.stattools to determine the ADF statistics of the resulting time series. Can you conclude from these statistics that the fractionally differentiated time series is stationary? (20 points)**

In [23]:
q2_norm_df = fracDiff_FFD(df[['close']], log_mind, thres=0.01).dropna()
q2_log_df = fracDiff_FFD(df[['log_close']], log_mind, thres=0.01).dropna()

In [31]:
q2_norm_adfuller = adfuller(q2_norm_df,maxlag=1,regression='c',autolag=None)
q2_log_adfuller = adfuller(q2_log_df,maxlag=1,regression='c',autolag=None)

In [29]:
q2_norm_adfuller

(-4.910394309077451,
 3.328556787885713e-05,
 1,
 3297,
 {'1%': -3.4323349555967604,
  '5%': -2.862417036057391,
  '10%': -2.5672368644181542})

In [32]:
q2_log_adfuller

(-3.756688620019477,
 0.0033828660986467266,
 1,
 3297,
 {'1%': -3.4323349555967604,
  '5%': -2.862417036057391,
  '10%': -2.5672368644181542})